In [3]:
from utils.game_map_class import GameMap

In [4]:
import networkx as nx

In [5]:
ex_blocks = \
    {"1":
         ("red",
          ["A",
           "B",
           "C",
           "D"]
         ),
     "2":
         ("blue",
          ["E",
           "F"]
         ),
     "3":
         ("green"
          ,["G",
            "H",
            "I"]
         )
    }

In [6]:
ex_edges = [("A","B"),
            ("B","C"),
            ("C","D"),
            ("D","A"),
            ("B","E"),
            ("C","E"),
            ("E","F"),
            ("F","G"),
            ("G","H"),
            ("G","I")]

In [7]:
# In a real game you'd get actual GameTeams for your dict, but for demonstration we won't:
ex_teams = {'red':None,'blue':None}

In [8]:
simple_map = GameMap(ex_blocks,ex_edges,ex_teams)

In [9]:
# I was thinking we could combine different evaluation functions

In [10]:
def evaluation_function_a(game_map):
    """This function takes a game map (i.e. a state) and returns a dictionary of team
    to % equity they have in the game
    
    This one just calculates proportion of edges it would win"""
    
    edge_win_dict = {}
    for team in game_map.teams:
        edge_win_dict[team] = 0

    total = 0
    for e in game_map.graph.edges():
        nodeA = game_map.graph.nodes[e[0]]
        nodeB = game_map.graph.nodes[e[1]]
        if nodeA['team'] or nodeB['team']:
            total += 1
            if nodeA['team'] == nodeB['team']:
                edge_win_dict[nodeA['team']] += 1
            elif nodeA['team'] == None or nodeA['num_troops'] < nodeB['num_troops']:
                edge_win_dict[nodeB['team']] += 1
            elif nodeB['team'] == None or nodeB['num_troops'] < nodeA['num_troops']:
                edge_win_dict[nodeA['team']] += 1
            else:
                total -= 1

    for team in game_map.teams:
        if total != 0:
            edge_win_dict[team] = edge_win_dict[team]/total

    return edge_win_dict
        

In [11]:
def EdgeWin(team, game_map):
    """This function takes a game map (i.e. a state) and returns
    the % equity they have in the game

    This one just calculates proportion of edges it would win"""

    edge_win_dict = {}
    for team_name in game_map.teams:
        edge_win_dict[team_name] = 0

    total = 0
    for e in game_map.graph.edges():
        nodeA = game_map.graph.nodes[e[0]]
        nodeB = game_map.graph.nodes[e[1]]
        if nodeA['team'] or nodeB['team']:
            total += 1
            if nodeA['team'] == nodeB['team']:
                edge_win_dict[nodeA['team']] += 1
            elif nodeA['team'] == None or nodeA['num_troops'] < nodeB['num_troops']:
                edge_win_dict[nodeB['team']] += 1
            elif nodeB['team'] == None or nodeB['num_troops'] < nodeA['num_troops']:
                edge_win_dict[nodeA['team']] += 1
            else:
                total -= 1

    if total == 0:
        equity = .5
    else:
        equity = edge_win_dict[team.getName()]/total

    return equity

In [12]:
evaluation_function_a(simple_map)
    

{'red': 0, 'blue': 0}

In [13]:
def make_graph(blocks,edges):
    graph = nx.Graph()
    for b in blocks.keys():
        for node in blocks[b][1]:
            graph.add_node(node,block=b,team=None,num_troops=0)
            graph.nodes[node]['neighbors'] = []

    for edge in edges:
        graph.add_edge(edge[0],edge[1])
        graph.nodes[edge[0]]['neighbors'].append(edge[1])
        graph.nodes[edge[1]]['neighbors'].append(edge[0])
    return graph

In [14]:
simple_map.graph.nodes['A']['team'] = 'red'

In [15]:
simple_map.graph.nodes['A']['num_troops'] = 3

In [16]:
evaluation_function_a(simple_map)

{'red': 1.0, 'blue': 0.0}

In [17]:
trial = GameMap(ex_blocks,ex_edges,{})

In [18]:
from utils.game_team_class import GameTeam

In [19]:
teams = []
team_names = ['red','blue']
for i in range(len(team_names)):
    strategy_class = None
    team_name = team_names[i]
    teams.append(GameTeam(team_name, trial, strategy = strategy_class, territories = []))

In [20]:
trial.teams

{'red': <utils.game_team_class.GameTeam at 0x7fa503f34358>,
 'blue': <utils.game_team_class.GameTeam at 0x7fa503f34390>}

In [22]:
import utils.map_setup_functions as msf

In [23]:
trial, teams = msf.setGameBoardRandom(team_names,trial)

In [24]:
evaluation_function_a(trial)

{'red': 0.75, 'blue': 0.25}

In [25]:
list(trial.graph.nodes(data=True))

[('A',
  {'block': '1', 'team': 'red', 'num_troops': 1, 'neighbors': ['B', 'D']}),
 ('B',
  {'block': '1',
   'team': 'red',
   'num_troops': 1,
   'neighbors': ['A', 'C', 'E']}),
 ('C',
  {'block': '1',
   'team': 'blue',
   'num_troops': 1,
   'neighbors': ['B', 'D', 'E']}),
 ('D',
  {'block': '1', 'team': 'red', 'num_troops': 1, 'neighbors': ['C', 'A']}),
 ('E',
  {'block': '2',
   'team': 'blue',
   'num_troops': 1,
   'neighbors': ['B', 'C', 'F']}),
 ('F',
  {'block': '2', 'team': 'red', 'num_troops': 1, 'neighbors': ['E', 'G']}),
 ('G',
  {'block': '3',
   'team': 'red',
   'num_troops': 1,
   'neighbors': ['F', 'H', 'I']}),
 ('H', {'block': '3', 'team': 'blue', 'num_troops': 1, 'neighbors': ['G']}),
 ('I', {'block': '3', 'team': 'blue', 'num_troops': 1, 'neighbors': ['G']})]

In [26]:
list(trial.graph.edges)

[('A', 'B'),
 ('A', 'D'),
 ('B', 'C'),
 ('B', 'E'),
 ('C', 'D'),
 ('C', 'E'),
 ('E', 'F'),
 ('F', 'G'),
 ('G', 'H'),
 ('G', 'I')]

In [28]:
import utils.heuristics as h

In [29]:
team = trial.teams['red']

In [30]:
h.BST_Heuristic(team,trial)

{'F': 1.0, 'G': 2.0, 'A': 0.0, 'D': 1.0, 'B': 2.0}

In [31]:
h.BSR_Heuristic(team,trial)

{'F': 0.16666666666666666,
 'G': 0.3333333333333333,
 'A': 0.0,
 'D': 0.16666666666666666,
 'B': 0.3333333333333333}

In [32]:
EdgeWin(trial.teams['blue'],trial)

0.25

In [33]:
EdgeWin(trial.teams['red'],trial)

0.75

In [35]:
trial.teams['red'].troops

5

In [36]:
import copy

In [37]:
new_trial = copy.deepcopy(trial)

In [40]:
new_trial.teams['red'].addTerritory('E')

In [42]:
trial.teams['red'].territories

['F', 'G', 'A', 'D', 'B']

In [43]:
new_trial.teams['red'].territories

['F', 'G', 'A', 'D', 'B', 'E']

In [44]:
trial.getTeam('A').territories

['F', 'G', 'A', 'D', 'B']

In [45]:
new_trial.getTeam('A').territories

['F', 'G', 'A', 'D', 'B', 'E']